# Предсказание новостных тематик
Задача предсказания тем новостей, в рамках открытого буткемпа "First Step in NLP: 2.0" от ФКН ВШЭ.

Тематики новостей:
* `Общество/Россия`: 0
* `Экономика`: 1
* `Силовые структуры`: 2
* `Бывший СССР`: 3
* `Спорт`: 4
* `Забота о себе`: 5
* `Строительство`: 6
* `Туризм/Путешествия`: 7
* `Наука и техника`: 8

Подключим Google-диск

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Установим и импортируем необходимые библиотеки

In [2]:
%%capture
!pip install selenium
!pip install pymorphy2

In [144]:
import os
import numpy as np
import pandas as pd
import re
import csv
import time
from datetime import datetime, timedelta

from bs4 import BeautifulSoup
import requests as rq
from selenium import webdriver
from selenium.webdriver.common.by import By

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
nltk.download("stopwords")
nltk.download("punkt")

import pymorphy2

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from imblearn.over_sampling import RandomOverSampler

from wordcloud import WordCloud

from IPython import display
from dataclasses import dataclass
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Определим константы

In [141]:
SLEEP = 1 # пауза между чтением url, в секундах
DEPTH = 10000 # количество ссылок на новости в каждой рубрике
PAGES = 500 # количество новостей в рубрике для парсинга

# путь к данным
DATA_DIR = '/'
SCRAPING_DIR = 'scraping/'
LINKS_SUBDIR = "links/"
NEWS_SUBDIR = "news/"
DATASET_DIR = 'dataset/'
TEST_DIR = 'test/'
SUBMIT_DIR = 'submit/'

YEAR_FROM = 2019
YEAR_TILL = 2023
MIN_TEXT_LEN = 10 # минимальная длина текстовых данных

RANDOM_STATE = 42 # параметр рандомизации

Опишем класс для парсинга новости

In [52]:
@dataclass
class Article:
    url: str = None
    title: str = None
    subtitle: str = None
    content: str = None
    datetime: str = None
    category: int = 0

Инициализируем web-скрейпер

In [53]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("headless")
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)

## Функции скрейпинга для источников новостей

### РИА Новости

Функция скачивания списка ссылок на новости выбранной тематики

In [54]:
def ria_get_list(scraper_id, base_url, idx, topic):

    links = []

    news_count = 20 # кол-во новостей на странице
    pages_count = DEPTH // news_count # кол-во итераций чтения страниц со ссылками

    url = base_url + topic # формируем ссылку на страницу рубрики
    print(url)
    driver.get(url) # открываем страницу
    time.sleep(SLEEP) # делаем паузу

    # кликаем по кнопке "загрузить еще"
    driver.execute_script(
        "document.getElementsByClassName('list-more')[0].click()"
    )
    time.sleep(1) # делаем паузу

    # скроллим страницу для автоматической подгрузки нужного количества новостей
    for i in tqdm(range(pages_count), leave=False):
        try:
            driver.execute_script(
                "window.scrollTo(0, document.body.scrollHeight - 1200)"
            )
            time.sleep(1) # делаем паузу
        except:
            pass

    # читаем ссылки на страницы новостей
    elems = driver.find_elements(By.XPATH, "//div[@class='list-item']//a[@class='list-item__title color-font-hover-only']")
    links = links + [elem.get_attribute('href') for elem in elems]

    return links

Функция парсинга страниц новостей

In [55]:
def ria_parse_page(url):

    # Create article data class object
    article = Article()

    # article url
    article.url = url

    # article id
    s = re.findall(r"\d+.html", article.url)[0]
    article_id = s[: s.find(".")]

    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article source
    source = article.url[8 : article.url.find(".")]

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find(
        "div",
        {
            "class": lambda x: x and (x.find(f"article m-article m-{source}") > -1),
            "data-article-id": article_id,
        },
    )

    if not obj:
        obj = soup.find(
            "div",
            {
                "class": lambda x: x and (x.find(f"article m-video m-{source}") > -1),
                "data-article-id": article.id,
            },
        )

    # process article title
    title = obj.find("div", {"class": "article__title"})
    title_2 = obj.find("h1", {"class": "article__title"})

    if title:
        article.title = title.text.strip()
    else:
        article.title = title_2.text.strip() if title_2 else ""

    # article subtitle
    subtitle = obj.find("h1", {"class": "article__second-title"})
    article.subtitle = subtitle.text.strip() if subtitle else ""

    # article content
    article.content = obj.find(
        "div", {"class": "article__body js-mediator-article mia-analytics"}
    ).text.strip()

    # article datetime
    article.datetime = obj.find("div", {"class": "article__info-date"}).find("a").text

    return article

### Фонтанка

In [56]:
def fontanka_get_list(scraper_id, base_url, idx, topic):

    links = []

    for year in range(YEAR_FROM, YEAR_TILL + 1):

        try:
            url = base_url + topic + f"arc/{year}/all.html" # формируем ссылку на страницу рубрики
            print(url)
            driver.get(url) # открываем общую страницу рубрики
            time.sleep(SLEEP) # делаем паузу

            # читаем дни недели
            days = driver.find_elements(
                By.XPATH,
                '//*[@id="app"]/div/div[4]/div[2]/div/section/div/div/div[2]/div[2]//a'
            )
            days_list = [d.get_attribute('href') for d in days]
            for day_url in tqdm(days_list, leave=False):
                try:
                    driver.get(day_url)
                    time.sleep(1) # делаем паузу

                    # читаем ссылки на страницы новостей
                    elems = driver.find_elements(
                        By.XPATH,
                        '//*[@id="app"]/div/div[4]/div[2]/div/section/div/div/div[2]/div/ul/li/div[2]/div[1]/a[1]'
                    )
                    links = links + [elem.get_attribute('href') for elem in elems]
                    r = re.compile("https://www.fontanka.ru/\d{4}/\d{2}/\d{2}/\d+/$")
                    links = list(filter(r.match, links))
                    if len(links) >= DEPTH:
                        break
                except:
                    pass
        except:
            pass

    return links

In [57]:
def fontanka_parse_page(url):

    # Create article data class object
    article = Article()

    # article url
    article.url = url

    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find("article")

    # process article title
    title = obj.find("h1")
    article.title = title.text.strip()

    # article subtitle
    article.subtitle = ""

    # article content
    article.content = obj.find(
        "section", {"itemprop": "articleBody"}
    ).text.strip()

    # article datetime
    article.datetime = obj.find(
        "span", {"itemprop": "datePublished"}
    ).text

    return article

### Коммерсант

In [58]:
def kommersant_get_list(scraper_id, base_url, idx, topic):

    links = []

    url = base_url + topic + '/month' # формируем ссылку на страницу рубрики
    print(url)
    driver.get(url) # открываем страницу

    while True:
      # кликаем по кнопке "загрузить еще"
      driver.execute_script(
          "document.getElementsByClassName('ui-button ui-button--standart ui-nav ui-nav--prev')[0].click()"
      )
      time.sleep(1) # делаем паузу
      # читаем ссылки на страницы новостей
      elems = driver.find_elements(By.XPATH, "//h2[@class='uho__name rubric_lenta__item_name']/a[@class='uho__link uho__link--overlay']")
      links = links + [elem.get_attribute('href') for elem in elems]
      if len(links) >= DEPTH:
          break

    time.sleep(SLEEP) # делаем паузу

    return links

In [59]:
def kommersant_parse_page(url):

    # Create article data class object
    article = Article()

    # article url
    article.url = url

    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find("div", {"class": "doc__body"})

    # process article title
    title = obj.find("h1")
    article.title = title.text.strip()

    # article subtitle
    article.subtitle = " "

    # article content
    article.content = obj.find(
        "div", {"class": "article_text_wrapper js-search-mark"}
    ).text.strip()

    # article datetime
    article.datetime = driver.find_element(
        By.XPATH, "//article/div[1]/time"
    ).text

    return article

### Лента

In [60]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])

        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 7) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')

        return out

In [61]:
def lenta_get_list(scraper_id, base_url, idx, topic):
    query = ''
    offset = 0
    size = 1000
    sort = "3"
    title_only = "0"
    domain = "1"
    material = "0"
    time_step = 7

    for year in range(YEAR_FROM, YEAR_TILL + 1):
        dateFrom = str(year) + '-01-01'
        dateTo = str(year) + '-01-31' # '-12-31'
        param_dict = {
                    'query'     : query,
                    'from'      : str(offset),
                    'size'      : str(size),
                    'dateFrom'  : dateFrom,
                    'dateTo'    : dateTo,
                    'sort'      : sort,
                    'title_only': title_only,
                    'type'      : material,
                    'bloc'      : topic,
                    'domain'    : domain
        }

        parser = lentaRu_parser()

        tbl = parser.get_articles(param_dict=param_dict, time_step=time_step)
        print('topic:', topic)

        tbl = tbl[['url', 'pubdate', 'title', 'text']]
        tbl = tbl.rename(columns={'pubdate': 'datetime', 'text': 'content'})
        tbl['subtitle'] = " "
        tbl['category'] = idx

        scraper_dir = DATA_DIR + SCRAPING_DIR + f"{scraper_id}/"

        if not os.path.exists(scraper_dir + NEWS_SUBDIR):
            os.mkdir(scraper_dir + NEWS_SUBDIR)

        news_file_path = scraper_dir + NEWS_SUBDIR + f"news_{idx}.csv"
        tbl.to_csv(news_file_path, sep=';', encoding='utf-8')

    return []

In [62]:
def lenta_parse_page(url):
    pass

## Парсинг новостей

Список источников новостей

In [63]:
scrapers = [
    [
        "ria", # id скрейпера
        "https://ria.ru/", # базовый url
        [
            "society/",# ссылки на тематические разделы
            "economy/",
            "",
            "",
            "",
            "",
            "tag_thematic_category_Stroitelstvo/",
            "tag_thematic_category_Turizm/",
            "science/"
        ],
        ria_get_list, # функция скачивания списка элементов выбранной тематики
        ria_parse_page # функция парсинга скачанных страниц
    ],
    [
        "fontanka",
        "https://www.fontanka.ru/",
        [
            "society/",
            "",
            "",
            "",
            "sport/",
            "",
            "stroy/",
            "turizm/",
            ""
        ],
        fontanka_get_list,
        fontanka_parse_page
    ],
    [
        "kommersant",
        "https://www.kommersant.ru/archive/",
        [
            "rubric/7",
            "rubric/3",
            "",
            "",
            "rubric/9",
            "",
            "",
            "",
            "online/296"
        ],
        kommersant_get_list,
        kommersant_parse_page
    ],
    [
        "lenta",
        "https://lenta.ru/",
        [
            "1",
            "4",
            "37",
            "3",
            "8",
            "87",
            "",
            "48",
            "5"
        ],
        lenta_get_list,
        lenta_parse_page
    ]
]

Сформируем списки ссылок на новости для каждой рубрики каждого из источников

In [ ]:
print('Scraping...')
print('')

for scraper in scrapers:
    scraper_id, base_url, topics, get_list, _ = scraper

    print(scraper_id)

    scraper_dir = DATA_DIR + SCRAPING_DIR + f"{scraper_id}/"
    if not os.path.exists(scraper_dir):
        os.mkdir(scraper_dir)
    if not os.path.exists(scraper_dir + LINKS_SUBDIR):
        os.mkdir(scraper_dir + LINKS_SUBDIR)

    for idx, topic in enumerate(topics):
        if topic != "":
            links = get_list(scraper_id, base_url, idx, topic)
            if len(links):
                links_file_path = scraper_dir + LINKS_SUBDIR + f"links_{idx}.csv"
                df_links = pd.DataFrame(data=links, columns=['link'])
                df_links.drop_duplicates(subset=['link'], inplace=True, ignore_index=True)
                df_links.to_csv(links_file_path, sep=';', encoding='utf-8')

    print('')

print('Scraping done.')

Скачаем новости для каждой рубрики

In [ ]:
print('Parsing...')
print('')

for scraper in scrapers:
    scraper_id, base_url, topics, _, parse_page = scraper

    print(scraper_id)

    scraper_dir = DATA_DIR + SCRAPING_DIR + f"{scraper_id}/"

    if not os.path.exists(scraper_dir + NEWS_SUBDIR):
        os.mkdir(scraper_dir + NEWS_SUBDIR)

    for idx, topic in enumerate(topics):
        if topic != "":
            links_file_path = scraper_dir + LINKS_SUBDIR + f"links_{idx}.csv"
            if os.path.exists(links_file_path):
                df_links = pd.read_csv(links_file_path, delimiter=';', usecols=['link'])
                links = df_links['link']

                news_file_path = scraper_dir + NEWS_SUBDIR + f"news_{idx}.csv"
                if os.path.exists(news_file_path):
                    df_cur_news = pd.read_csv(news_file_path, delimiter=';')
                    df_new_links = df_links.loc[~links.isin(df_cur_news['url'])]
                    df_new_links.reset_index(inplace=True)
                    new_links = df_new_links['link']
                    news_file_exists = True
                else:
                    new_links = links
                    news_file_exists = False
                if len(new_links) > 0:
                    print('topic ', idx, ':', len(new_links), 'new links')

                    data = []
                    for link in tqdm(new_links[:PAGES], leave=False):
                        try:
                            res = parse_page(link)
                            res.category = idx
                            data.append(res)
                        except:
                            pass
                    df = pd.DataFrame(data=data)

                    if news_file_exists:
                        df = pd.concat([df_cur_news, df], axis=0, ignore_index=True)
                        df.drop_duplicates(subset=['url'], inplace=True, ignore_index=True)

                    df.to_csv(news_file_path, sep=';', encoding='utf-8')
    print('')

print('Parsing done.')

Для каждого источника объединим полученные новости в общий файл

In [ ]:
print('Concatination...')
print('')

for scraper in scrapers:
    scraper_id, base_url, topics, _, _ = scraper

    scraper_dir = DATA_DIR + SCRAPING_DIR + f"{scraper_id}/"

    df_list = []

    for idx, topic in enumerate(topics):
        if topic != "":
            news_file_path = scraper_dir + NEWS_SUBDIR + f"news_{idx}.csv"
            df = pd.read_csv(
                news_file_path,
                delimiter=';',
                usecols=['url', 'title', 'subtitle', 'content', 'category'])
            df_list.append(df)

    df_news = pd.concat(df_list, axis=0, ignore_index=True)
    df_news = df_news.fillna('')

    print(scraper_id, ':', len(df_news), 'news')

    df_news.to_csv(scraper_dir + "news.csv", sep=';', encoding='utf-8')

print('')
print('Concatination done.')

Объединим все файлы новостей в один датасет

In [20]:
df_list = []

for scraper in scrapers:
    scraper_id, base_url, topics, _, _ = scraper

    scraper_dir = DATA_DIR + SCRAPING_DIR + f"{scraper_id}/"
    df_news = pd.read_csv(scraper_dir + "news.csv", delimiter=';', usecols=['url', 'title', 'subtitle', 'content', 'category'])
    df_list.append(df_news)

df_news_all = pd.concat(df_list, axis=0, ignore_index=True)
df_news_all.drop_duplicates(subset=['url'], inplace=True, ignore_index=True)

Объединим текстовые поля датафрейма

In [21]:
df_news_all['content'] = df_news_all['title'] + '. ' + df_news_all['subtitle'] + '. ' + df_news_all['content']
df_news_all.content = df_news_all.content.astype(str)
df_news_all['content'] = [re.sub('[\.\?!]+(\.) ', '\\1 ', t) for t in df_news_all['content']]
df_news_all = df_news_all.drop(columns=['title', 'subtitle'])

In [ ]:
df_news_all.sample(5)

Сохраним результат в файл

In [ ]:
all_news_path = DATA_DIR + SCRAPING_DIR + "news.csv"
df_news_all.drop('url', axis=1).to_csv(all_news_path, sep=';', encoding='utf-8')

print('News file:', all_news_path)
print('Total news:', len(df_news_all))

## Предобработка текстовых данных

Прочитаем CSV-файл и загрузим данные в датафрейм

In [24]:
news_file = DATA_DIR + SCRAPING_DIR + "news.csv"
df = pd.read_csv(news_file, usecols=['content', 'category'], delimiter=';')

Проверим данные на наличие пропусков

In [ ]:
df.isnull().sum()

Посчитаем распределение записей по категориям

In [ ]:
df['category'].value_counts()

In [ ]:
df['category'].value_counts(normalize=True)

In [ ]:
sns.set(rc={'figure.figsize':(15, 5)})
sns.countplot(x='category', data=df)
plt.xlabel("category", size = 12)
plt.ylabel("count", size = 12)
plt.show()

### Функции подготовки данных

Удаление пустых значений

In [29]:
def remove_empty(df):
    df.dropna(inplace=True)
    return df

Строковый тип колонок `title` и `content`, категориальный тип колонки `category`

In [30]:
def set_category_type(df):
    df = df.astype({'content': str, 'category': 'category'})
    return df

Токенизация текста (процесс разбиения текстового документа на отдельные фразы)

In [31]:
def tokenize_text(text):
    sents = sent_tokenize(text, language='russian')
    return sents

Токенизация фраз (процесс разбиения фраз на отдельные слова)

In [32]:
def tokenize_sent(sent):
    words = word_tokenize(sent, language='russian')
    return words

In [33]:
def tokenize_sents(sents):
    words = []
    for s in sents:
        w = tokenize_sent(s)
        words = words + w
    return words

Удаление ссылок

In [34]:
def remove_urls(text):
    text = re.sub(r"https?://[^,\s]+,?", "", text)
    return text

Стемминг, лемматизация, стоп-слова

In [35]:
# таблица пакетной замены символов
PUNCTUATION_STRING = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~«»'#-
punct_str = str.maketrans('', '', PUNCTUATION_STRING)

# алгоритм, используемый для лемматизации (приведения к нормальной форме) слов
morph = pymorphy2.MorphAnalyzer()

# алгоритм стемминга (нахождения основы) слов
stemmer = SnowballStemmer(language="russian")

# стоп-слова
stop_words = stopwords.words("russian")

stop_words.extend(['который', 'которая', 'также', 'такой', 'однако', 'это'])

Стемминг слов

In [36]:
def stemming(word):
    word = stemmer.stem(word)
    return word

Лемматизация слов

In [37]:
def lemmatize(word):
    word = morph.parse(word)[0].normal_form
    return word

Удаление цифр и знаков пунктуации

In [38]:
def clean(word):
    word = ''.join([x for x in word if not x.isdigit()])
    word = ''.join([x.translate(punct_str) for x in word])
    return word

Визуализация с помощью "облака слов"

In [39]:
def get_corpus(data):
    corpus = []
    for phrase in data:
        for word in phrase.split():
            corpus.append(word)
    return corpus

def str_corpus(corpus):
    str_corpus = ' '.join(corpus)
    return str_corpus

def get_wordCloud(corpus):
    word_сloud = WordCloud(background_color='white', width=800, height=600, max_words=100).generate(str_corpus(corpus))
    return word_сloud

def get_top_words(text):
    word_cloud = get_wordCloud(text)
    return word_cloud.words_.keys()

def show_wordCloud(text):
    word_cloud = get_wordCloud(text)
    fig = plt.figure(figsize=(30, 10))
    plt.subplot(1, 2, 1)
    plt.axis('off')
    plt.imshow(word_cloud)

Функция предобработки текста (на входе - текст, на выходе - список обработанных слов):

* приведение к нижнему регистру
* удаление чисел и знаков пунктуации
* исключение стоп-слов
* лемматизация / стемминг слов
* исключение дополнительных стоп-слов

In [40]:
def text_to_words_preprocessing(
        text, # текстовая строка
        transform=True, # флаг обработки слов
        mode='lemma', # тип нормализации слов: 'lemma' (по умолчанию), 'stem'
        add_stop_words=[] # список дополнительных стоп-слов
    ):
    text = text.replace('-', ' ')
    sents = tokenize_text(text)
    words = tokenize_sents(sents)
    words = [x.lower() for x in words]
    words = [clean(x) for x in words]
    words = [x for x in words if len(x) > 2]
    if transform:
        words = [x for x in words if x not in stop_words]
        if mode == 'lemma':
            words = [lemmatize(x) for x in words]
        elif mode == 'stem':
            words = [stemming(x) for x in words]
        if len(add_stop_words):
            words = [x for x in words if x not in add_stop_words]
    return words

Функция подготовки датасета

In [145]:
def data_preprocessing(
        df,
        transform=True, # флаг обработки слов
        aug=False, # флаг аугментации текста
        mode='lemma', # тип нормализации слов: lemma (по умолчанию), stem
        local_stop_words=[] # список дополнительных стоп-слов
    ):

    # дополнение, лемматизация/стемминг стоп-слов
    stop_words_set = stop_words.copy()
    stop_words_set.extend(local_stop_words)
    if mode == 'lemma':
        stop_words_set = set([lemmatize(x) for x in stop_words_set])
    elif mode == 'stem':
        stop_words_set = set([stemming(x) for x in stop_words_set])

    # предобработка текста
    df['text'] = [
        ' '.join(text_to_words_preprocessing(
            x,
            mode=mode,
            transform=transform,
            add_stop_words=stop_words_set
            )
        ) for x in tqdm(df['content'], leave=False)
    ]

    # удаляем пустые значения
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = remove_empty(df)

    # удаляем элементы, где недостаточно текста
    df[df['text'].str.len() >= MIN_TEXT_LEN]

    return df

Подготовим загруженные данные

In [42]:
df = remove_empty(df) # удаляем пустые значения
df = set_category_type(df) # присваиваем типы колонкам

Проанализируем состав текста до обработки

In [ ]:
show_wordCloud(df['content'])

Перечислим стоп-слова, специфические для данного датасета

In [44]:
local_stop_words = [
    'январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь',
    'новости', 'агентство', 'фото', 'фотохост', 'фотохостагентство', 'москва', 'риа', 'фонтанка', 'пресс', 'прессслужба',
    'поделиться', 'перейти', 'istockcom', 'медиабанк'
]

## Формирование датасета

In [ ]:
data = data_preprocessing(df, mode='lemma', transform=True, local_stop_words=local_stop_words)
data = data.drop(columns=['content'])

dataset_path = DATA_DIR + DATASET_DIR + 'news_data.csv'

data = pd.DataFrame(data)
data.to_csv(dataset_path, sep=';')

print('Dataset file:', dataset_path)
print('Total news:', len(data))

Проанализируем состав текста после обработки

In [ ]:
show_wordCloud(data['text'])

Формируем и сохраняем тестовый датасет

In [ ]:
test_path = DATA_DIR + TEST_DIR + 'test_news.csv'

df_test = pd.read_csv(test_path, usecols=['content'])
df_test = df_test[:200].astype({'content': str})
df_test = remove_empty(df_test)

df_test.head()

In [ ]:
df_test = data_preprocessing(df_test, mode='lemma', local_stop_words=local_stop_words)
df_test = pd.DataFrame(df_test)
df_test = df_test.drop(columns=['content'])
df_test = df_test.replace(r'^\s*$', np.nan, regex=True)
df_test = remove_empty(df_test)

In [ ]:
dataset_test_path = DATA_DIR + DATASET_DIR + 'test_data.csv'

df_test.to_csv(dataset_test_path, sep=',')

print('Test dataset file:', dataset_test_path)
print('Total news:', len(df_test))

## Обучение модели

Функция присвоения строкового типа колонке `text`, категориального типа колонке `category`

In [68]:
def set_category_type(df):
    df = df.astype({'text': str, 'category': 'category'})
    return df

Читаем датасет для обучения

In [70]:
dataset_path = DATA_DIR + DATASET_DIR + 'news_data.csv'
dataset = pd.read_csv(dataset_path, delimiter=';', usecols=['text', 'category'])

Список категорий

In [71]:
category_set = dataset['category'].unique()
category_set.sort()

Формируем тренировочный датасет

In [99]:
df = set_category_type(dataset)

Настройки балансировки датасета

In [ ]:
total_len = 350000 # общий размер датасета
balance_weigths = [0.48, 0.11, 0.07, 0.12, 0.06, 0.03, 0.04, 0.04, 0.05] # веса категорий

print(sum(balance_weigths))

In [ ]:
balance_set = [round(x * total_len) for x in balance_weigths]
balance_size = dict(enumerate(balance_set)) # размеры датасета по пубрикам

print(balance_size)

Для балансировки датасета используем класс RandomOverSampler из библиотеки imbalanced-learn

In [100]:
df_tmp = pd.DataFrame()
for i in range(len(balance_size)):
  x = df[df['category'] == i][:balance_set[i]]
  a = pd.DataFrame(data=x)
  df_tmp = pd.concat([df_tmp, a], ignore_index=True)

In [101]:
over_sampler = RandomOverSampler(sampling_strategy=balance_size, random_state=RANDOM_STATE)
df, _ = over_sampler.fit_resample(df_tmp, df_tmp['category'])

Распределение записей по категориям после балансировки

In [ ]:
sns.set(rc={'figure.figsize':(15, 5)})
sns.countplot(x='category', data=df)
plt.xlabel("category", size = 12)
plt.ylabel("count", size = 12)
plt.show()

Перемешиваем датасет

In [103]:
df = df.sample(frac=1)

Разделим данные на тренировочную и тестовую выборки

In [104]:
X_train, X_test, y_train, y_test = train_test_split(df, df['category'], train_size=0.8, random_state=RANDOM_STATE, stratify=df['category'])

X_train = X_train['text']
X_test = X_test['text']

Функция вывода матрицы соответствий

In [81]:
def cm_plot(y_test, y_pred, labels=None):
    sns.set(font_scale=1.3)
    cm = confusion_matrix(y_test, y_pred, labels=labels)
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(cmn, fmt='.2f', cmap='Blues', annot=True, cbar=False, xticklabels=labels, yticklabels=labels)
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    return plt.show()

Выберем векторизатор и классификатор

In [85]:
def get_vectorizer(v, params):
    if v == 'tfidf':
        vectorizer = TfidfVectorizer(**params)
    else:
        return False

    return vectorizer

In [113]:
def get_classifier(c):
    if c == 'LGR':
        clf = LogisticRegression(random_state=RANDOM_STATE)
        parameters = {}
    elif c == 'SVC':
        clf = LinearSVC(random_state=RANDOM_STATE, max_iter=10000, multi_class='crammer_singer')
        parameters = {}
    elif c == 'SGD':
        clf = SGDClassifier(random_state=RANDOM_STATE, n_iter_no_change=5)
        parameters = {}
    else:
        return False


    return clf, parameters

Функция проведения экспериментов

In [121]:
def do_experiments():
    model_names = []
    model_best_params = {}
    model_estim = {}
    model_best_metric = {}

    for c in classifiers:
        model_estim[c] = []
        model_best_metric[c] = 0

    for v in vectorizers:
        params_set = [
            {'ngram_range': (1, 2), 'max_df': 0.1, 'min_df': 1},
            {'ngram_range': (1, 2), 'max_df': 0.1, 'min_df': 3},
            {'ngram_range': (1, 2), 'max_df': 0.1, 'min_df': 5},
            {'ngram_range': (1, 2), 'max_df': 0.5, 'min_df': 1},
            {'ngram_range': (1, 2), 'max_df': 0.5, 'min_df': 3},
            {'ngram_range': (1, 2), 'max_df': 0.5, 'min_df': 5},
            {'ngram_range': (1, 2), 'max_df': 0.9, 'min_df': 1},
            {'ngram_range': (1, 2), 'max_df': 0.9, 'min_df': 3},
            {'ngram_range': (1, 2), 'max_df': 0.9, 'min_df': 5}
        ]

        for vect_params in params_set:
            vectorizer = get_vectorizer(v, vect_params)
            vector_train = vectorizer.fit_transform(X_train)

            model_name = vect_params
            model_names.append(model_name)

            for c in classifiers:

                clf, clf_parameters = get_classifier(c)
                clf.fit(vector_train, y_train)

                vector_test = vectorizer.transform(X_test)

                prediction = clf.predict(vector_test)
                model_metric = accuracy_score(prediction, y_test)

                print(c, model_metric, vect_params)

                model_estim[c].append(model_metric)

                if model_metric > model_best_metric[c]:
                    model_best_metric[c] = model_metric
                    model_best_params[c] = [vect_params]

    return model_names, model_estim, model_best_params

### Проведение экспериментов

In [115]:
vectorizers = ['tfidf']
classifiers = ['SVC', 'SGD']

In [ ]:
datasets_list, estimation, best_params = do_experiments()

In [ ]:
clfs = estimation.keys()
clfs_score = {}
width = 1 / (len(clfs) + 1)
x_shift = (len(clfs) - 1) / 2
y_min = 1
y_max = 0

x = np.arange(len(datasets_list))
sns.set_theme()
sns.set(rc={'figure.figsize':(15,4)})

colors = ['#ff6b81', '#1e90ff', '#2ed573']

fig, ax = plt.subplots()
i = 0
for clf_id in clfs:
  ax.bar(x + (i - x_shift) * width, estimation[clf_id], width, label=clf_id, color=colors[i])
  i += 1
  y_min = min(y_min, min(estimation[clf_id]))
  y_max = max(y_max, max(estimation[clf_id]))
  clfs_score[clf_id] = max(estimation[clf_id])
ax.set_xticks(x)
ax.set_xticklabels(datasets_list)
ax.set_ylim(y_min * 0.99, y_max * 1.005)
ax.legend()
plt.title('Метрика Accuracy')

Выберем лучший классификатор

In [ ]:
best_clf = max(clfs_score, key=clfs_score.get)
print('Best classifier:', best_clf)

Выполним векторизацию с использованием лучших параметров

In [ ]:
best_params_max = best_params[best_clf][0]
print('Best parameters:', best_params_max)

In [127]:
vectorizer = get_vectorizer('tfidf', best_params_max)
vector_train = vectorizer.fit_transform(X_train)

Выполним обучение и тестирование

In [ ]:
clf, clf_parameters = get_classifier(best_clf)
clf.fit(vector_train, y_train)

Выполним предсказание на тестовых данных

In [129]:
vector_test = vectorizer.transform(X_test)
y_pred = clf.predict(vector_test)

In [ ]:
model_metric = accuracy_score(y_pred, y_test)
print('Accuracy:', model_metric)

Выведем матрицу соответствий

In [ ]:
cm_plot(y_test, y_pred, labels=category_set)

## Тестовое предсказание

Обучим модель на всех данных

In [132]:
X_train_all = df['text']
y_train_all = df['category']

In [133]:
vector_train_all = vectorizer.fit_transform(X_train_all)

In [ ]:
clf.fit(vector_train_all, y_train_all)

In [ ]:
df_news_test = pd.read_csv(dataset_test_path, sep=';')

df_test.head()

In [136]:
vector_news_test = vectorizer.transform(df_test['text'])
y_pred_news_test = clf.predict(vector_news_test)
df_test['topic'] = y_pred_news_test

Сохраняем данные для отправки на конкурс


In [ ]:
submit_path = DATA_DIR + SUBMIT_DIR + "submission.csv"

df_test['topic'].to_csv(submit_path, sep=',', index_label='index')

print('Submission file:', submit_path)